1. Escriba el código de SQL que le permite conocer el monto y la cantidad de las transacciones que SIMETRIK considera como conciliables para la base de CLAP

In [ ]:
WITH CTE AS (
SELECT 
	*,
	ROW_NUMBER() OVER(PARTITION BY ID_BANCO ORDER BY FECHA_RECEPCION_BANCO DESC) AS contador  
	FROM CLAP --Aca consigo si hay duplicados para mismo ID
),
CTE2 AS (

SELECT 
		A.ID_BANCO,
		CAST(A.MONTO*1.0 AS NUMERIC(10,2)) AS CLAP_MONTO

	FROM CTE a

	INNER JOIN BANSUR B ON A.ID_BANCO = B.ID_ADQUIRIENTE
	AND CONCAT(coalesce(INICIO6_TARJETA,0),FINAL4_TARJETA) = B.TARJETA
	AND CAST(A.FECHA_TRANSACCION AS DATE) = B.FECHA_TRANSACCION
	AND (CAST(A.MONTO*1.0 AS NUMERIC(10,2)))-B.MONTO <= 0.99
	WHERE A.contador = 1 -- Con esto consigo el ID mas nuevo
	AND A.TIPO_TRX = 'PAGADA'-- Con esto solo los activos y elimino los Nulls
	AND B.TIPO_TRX =  'PAGO'
	)
SELECT COUNT(ID_BANCO) AS QTY_TRANSACTION, SUM(CLAP_MONTO) AS SUM_TRANSACTION
FROM CTE2;


Para esta primera pregunta se hicieron varias validaciones que eran solicitadas. En primer lugar, en caso que hubiese mismo ID para distintos estados, elegir el más nuevo. 
Luego, se refina pagos que:
- Estén Pagos
- Id coincida en las DB Clap y DB Bansur
- Monto coincida
- Dígitos de la tarjeta crédito coincidan en ambas bases de datos.

2. Escriba el código de SQL que le permite conocer el monto y la cantidad de las transacciones que SIMETRIK considera como conciliables para la base de BANSUR

In [ ]:
WITH CTE AS (

SELECT 
	*,
	ROW_NUMBER() OVER(PARTITION BY ID_ADQUIRIENTE order by FECHA_RECEPCION desc) as contador  
	from BANSUR
	),
CTE2 AS(

SELECT A.MONTO,A.ID_ADQUIRIENTE
FROM CTE A
	INNER JOIN CLAP B ON A.ID_ADQUIRIENTE = B.ID_BANCO 
	AND  A.TARJETA = CONCAT(coalesce(INICIO6_TARJETA,0),FINAL4_TARJETA)
	AND A.FECHA_TRANSACCION = CAST(B.FECHA_TRANSACCION AS DATE)
	AND B.MONTO-(CAST(A.MONTO*1.0 AS NUMERIC(10,2))) <=0.99
	WHERE A.contador = 1
	AND A.TIPO_TRX = 'PAGO'
	AND B.TIPO_TRX = 'PAGADA'
)

SELECT SUM(MONTO) AS SUM_TRANSACTION ,COUNT(ID_ADQUIRIENTE) AS QTY_TRANSACTION FROM CTE2;


La aproximación fue similar a la pregunta anterior.

3. Teniendo en cuenta los criterios de cruce entre ambas bases conciliables, escriba una sentencia de SQL que contenga la información de CLAP y BANSUR; agregue una columna en la que se evidencie si la transacción cruzó o no con su contrapartida y una columna en la que se inserte un ID autoincremental para el control de la conciliación

In [ ]:
WITH CTE AS (
SELECT 
	*,
	COUNT(ID_BANCO) OVER(PARTITION BY ID_BANCO ORDER BY FECHA_RECEPCION_BANCO DESC) AS contador
	FROM CLAP a

),
CTE2 AS (

SELECT 
		A.ID_BANCO,
		CAST(A.MONTO*1.0 AS NUMERIC(10,2)) AS CLAP_MONTO
		

	FROM CTE a

	INNER JOIN BANSUR B ON A.ID_BANCO = B.ID_ADQUIRIENTE
	AND CONCAT(coalesce(INICIO6_TARJETA,0),FINAL4_TARJETA) = B.TARJETA
	AND CAST(A.FECHA_TRANSACCION AS DATE) = B.FECHA_TRANSACCION
	AND (CAST(A.MONTO*1.0 AS NUMERIC(10,2)))-B.MONTO <= 0.99
	WHERE A.contador = 1
	AND A.TIPO_TRX = 'PAGADA'
	AND B.TIPO_TRX =  'PAGO'
	)
SELECT ID_BANCO 
INTO #CLAP_TEMP
FROM  CTE2;


CREATE TABLE #BANSUR_TEMP (
ID INT IDENTITY,
ID_BANSUR BIGINT,
TARJETA_BANSUR BIGINT,
FECHA_RECEPCION DATE,
CONCILIABLE CHAR(2)
);

INSERT INTO #BANSUR_TEMP (ID_BANSUR,TARJETA_BANSUR,FECHA_RECEPCION)
SELECT ID_ADQUIRIENTE,TARJETA,FECHA_RECEPCION FROM BANSUR;


UPDATE A
	SET CONCILIABLE = 'SI' FROM #BANSUR_TEMP A
	INNER JOIN #CLAP_TEMP B ON A.ID_BANSUR = B.ID_BANCO

UPDATE #BANSUR_TEMP 
	SET CONCILIABLE = 'NO'
	WHERE CONCILIABLE IS NULL;

--VER RESULTADOS
SELECT * FROM #BANSUR_TEMP


Mi razonamiento aquí fue de esta manera:

- Obtengamos esos ID_BANCO que hicieron MATCH con las validaciones que hicimos anteriormente y luego, si están presentes en la base de BANSUR le ponemos un ‘flag’.

El procedimiento fue:
- Guardar ID con match en tabla temporal #CLAP_TEMP.
- Creación tabla temporal #BANSUR_TEMP con algunas columnas relevantes, pero todas las filas.
- Update con flag, uniendo #CLAP_TEMPORAL.
- Finalmente para prolijidad se llena con ‘no’ donde no haya coincidencia.

Espero que te haya gustado tanto como a mí! Si tenes comentarios no dudes en escribirme!
